In [ ]:
# Set environment seed
import os
os.environ['PYTHONHASHSEED']=str(1)

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import re
from tqdm import tqdm
import matplotlib.pyplot as plt
import os
import datetime
import json
from contextlib import redirect_stdout
import plotly.graph_objects as go
import plotly.express as px
from sklearn.model_selection import train_test_split
import time

from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
def reset_random_seeds():
    '''
    Sets all necessary seed for reproduceability.
    '''
    os.environ['PYTHONHASHSEED']=str(1)
    tf.random.set_seed(1)
    np.random.seed(1)
    
reset_random_seeds()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
data = pd.read_csv('https://drive.google.com/uc?id=13skTlmf1OiiM8sN6eoLPTToJu5zfdzui&export=download')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
x_train, x_test, valid_train, valid_test = train_test_split(data['review'], data['sentiment'], 
                                                    test_size=0.2, random_state=0, 
                                                    stratify=data['sentiment'])

# Split X_test and valid_test store as Y_test, Y_true. This will be used for separate evaluation of model
Y_true = valid_test[int(len(valid_test)/4)*3::] # Set Y_true = 2nd 25% of labels from test set (Ground truth for evaluation)
Y_test = x_test[int(len(x_test)/4)*3::]# Set Y_test = 2nd 25% of rows from test set

x_test = x_test[0:int(len(x_test)/4)*3] # Set X_test to the 1st 25% of X_test
valid_test = valid_test[0:int(len(valid_test)/4)*3] # Set valid_test to the 1st 25% of valid_test

In [ ]:
# Set the hyper parameters
hyper_params = {
    'VOCAB_SIZE':50000,
    'EPOCHS':80,
    'BS':512,
    'LR':0.01,
    'OOV_TOK':""
}

In [ ]:
# Tokenize on the training dataset and apply it on our training and testing dataset

tokenizer = Tokenizer(num_words = hyper_params['VOCAB_SIZE'], oov_token=hyper_params['OOV_TOK'])
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)

In [ ]:
# visualize the statistics for the number of words in a sentence for any sentence in our training dataset 

num_words = []
for item in (train_sequences):
    num_words.append(len(item))
fig = px.box(num_words)
fig.show()

In [ ]:

# We will set the max length to 520, the upper fence value.

hyper_params['MAX_LENGTH'] = 520
hyper_params['PADDING_TYPE'] = "post"
hyper_params['TRUNC_TYPE'] = "post"

In [ ]:
# perform padding on the training and testing dataset

training_padded = pad_sequences(train_sequences, padding=hyper_params['PADDING_TYPE'], 
                                maxlen=hyper_params['MAX_LENGTH'], truncating=hyper_params['TRUNC_TYPE'])
testing_padded = pad_sequences(test_sequences, padding=hyper_params['PADDING_TYPE'], 
                               maxlen=hyper_params['MAX_LENGTH'], truncating=hyper_params['TRUNC_TYPE'])

In [ ]:
# Change categorical data to numerical

train_labels = []
test_labels = []

for item in valid_train:
    if item == 'positive':
        train_labels.append(1)
    else:
        train_labels.append(0)
        
for item in valid_test:
    if item == 'positive':
        test_labels.append(1)
    else:
        test_labels.append(0)

train_labels = np.asarray(train_labels).astype('float32')
test_labels = np.asarray(test_labels).astype('float32')

#GRU-1 RNN Model


We will construct a 1-layer GRU to observe its behaviour.

In [ ]:
# Create a "basic

GRU1_model = keras.Sequential([
    keras.layers.Embedding(len(word_index)+1, 20, input_length=520),
    keras.layers.GRU(100),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

GRU1_model.compile(optimizer=keras.optimizers.Adam(learning_rate=hyper_params['LR']),
                 loss='binary_crossentropy', metrics=['accuracy'])

GRU1_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 520, 20)           2244520   
                                                                 
 gru (GRU)                   (None, 100)               36600     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 2,281,221
Trainable params: 2,281,221
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train the basic 1-layer GRU

GRU1_history = GRU1_model.fit(training_padded, train_labels, batch_size=hyper_params['BS'], 
                        epochs=hyper_params['EPOCHS'], validation_data=(testing_padded, test_labels))

Epoch 1/80
79/79 [==============================] - 9s 71ms/step - loss: 0.6939 - accuracy: 0.5034 - val_loss: 0.6935 - val_accuracy: 0.4979
Epoch 2/80
79/79 [==============================] - 5s 61ms/step - loss: 0.6884 - accuracy: 0.5157 - val_loss: 0.7093 - val_accuracy: 0.5051
Epoch 3/80
79/79 [==============================] - 5s 58ms/step - loss: 0.6772 - accuracy: 0.5191 - val_loss: 0.7013 - val_accuracy: 0.5011
Epoch 4/80
79/79 [==============================] - 5s 58ms/step - loss: 0.6620 - accuracy: 0.5304 - val_loss: 0.7214 - val_accuracy: 0.4972
Epoch 5/80
79/79 [==============================] - 5s 59ms/step - loss: 0.6539 - accuracy: 0.5354 - val_loss: 0.7371 - val_accuracy: 0.5011
Epoch 6/80
79/79 [==============================] - 5s 59ms/step - loss: 0.7185 - accuracy: 0.5190 - val_loss: 0.6951 - val_accuracy: 0.5243
Epoch 7/80
79/79 [==============================] - 5s 59ms/step - loss: 0.6820 - accuracy: 0.5418 - val_loss: 0.6939 - val_accuracy: 0.5253
Epoch 8/80
79

In [ ]:
# Save loss figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=GRU1_history.epoch, y=GRU1_history.history['loss'], name='Training Loss'))
fig.add_trace(go.Scatter(x=GRU1_history.epoch, y=GRU1_history.history['val_loss'], name='Validation Loss'))

fig.show()
fig.write_html('basic_GRU1_loss.html')
#fig.write_image('basic_GRU1_loss.jpeg')

In [ ]:
# Save accuracy figure

fig = go.Figure()
fig.add_trace(go.Scatter(x=GRU1_history.epoch, y=GRU1_history.history['accuracy'], name='Training Accuracy'))
fig.add_trace(go.Scatter(x=GRU1_history.epoch, y=GRU1_history.history['val_accuracy'], name='Validation Accuracy'))

fig.show()
fig.write_html('basic_GRU1_acc.html')
#fig.write_image('basic_GRU1_acc.jpeg')

In [ ]:
# Save history as csv and json

# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(GRU1_history.history) 

# save to csv:
hist_csv_file = 'history_GRU1_basic.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

#GRU-2 RNN Model


We will construct a 2-layer GRU to observe its behaviour. 

In [ ]:
# Create a "basic" 2-layer GRU


GRU2_model = keras.Sequential([
    keras.layers.Embedding(len(word_index)+1, 20, input_length=520),
    keras.layers.GRU(100, return_sequences=True),
    keras.layers.Dropout(0.5),
    keras.layers.GRU(100),
    keras.layers.Dropout(0.5),    
    keras.layers.Dense(1, activation='sigmoid')
])

GRU2_model.compile(optimizer=keras.optimizers.Adam(learning_rate=hyper_params['LR']),
                 loss='binary_crossentropy', metrics=['accuracy'])

GRU2_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 520, 20)           2244520   
                                                                 
 gru_1 (GRU)                 (None, 520, 100)          36600     
                                                                 
 dropout_1 (Dropout)         (None, 520, 100)          0         
                                                                 
 gru_2 (GRU)                 (None, 100)               60600     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 2,341,821
Trainable params: 2,341,821
No

In [ ]:
# Train the basic 2-layer GRU model

GRU2_history = GRU2_model.fit(training_padded, train_labels, batch_size=hyper_params['BS'], 
                        epochs=hyper_params['EPOCHS'], validation_data=(testing_padded, test_labels))

Epoch 1/80
79/79 [==============================] - 14s 143ms/step - loss: 0.6948 - accuracy: 0.5009 - val_loss: 0.6942 - val_accuracy: 0.5007
Epoch 2/80
79/79 [==============================] - 11s 137ms/step - loss: 0.6944 - accuracy: 0.5038 - val_loss: 0.6981 - val_accuracy: 0.5033
Epoch 3/80
79/79 [==============================] - 11s 138ms/step - loss: 0.6804 - accuracy: 0.5210 - val_loss: 0.6992 - val_accuracy: 0.5017
Epoch 4/80
79/79 [==============================] - 11s 138ms/step - loss: 0.6639 - accuracy: 0.5293 - val_loss: 0.7360 - val_accuracy: 0.4992
Epoch 5/80
79/79 [==============================] - 11s 138ms/step - loss: 0.6556 - accuracy: 0.5293 - val_loss: 0.7430 - val_accuracy: 0.5001
Epoch 6/80
79/79 [==============================] - 11s 140ms/step - loss: 0.6732 - accuracy: 0.5267 - val_loss: 0.7229 - val_accuracy: 0.5016
Epoch 7/80
79/79 [==============================] - 11s 140ms/step - loss: 0.6555 - accuracy: 0.5336 - val_loss: 0.7437 - val_accuracy: 0.5003

In [ ]:
# Save loss figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=GRU2_history.epoch, y=GRU2_history.history['loss'], name='Training Loss'))
fig.add_trace(go.Scatter(x=GRU2_history.epoch, y=GRU2_history.history['val_loss'], name='Validation Loss'))

fig.show()
fig.write_html('basic_GRU2_loss.html')
#fig.write_image('basic_GRU2_loss.jpeg')

In [ ]:
# Save accuracy figure

fig = go.Figure()
fig.add_trace(go.Scatter(x=GRU2_history.epoch, y=GRU2_history.history['accuracy'], name='Training Accuracy'))
fig.add_trace(go.Scatter(x=GRU2_history.epoch, y=GRU2_history.history['val_accuracy'], name='Validation Accuracy'))

fig.show()
fig.write_html('basic_GRU2_acc.html')
#fig.write_image('basic_GRU2_acc.jpeg')

In [ ]:
# Save history as csv and json

# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(GRU2_history.history) 

# save to csv:
hist_csv_file = 'history_GRU2_basic.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

#LSTM-1 RNN Model


We will construct a 1-layer LSTM to observe its behaviour. 

In [ ]:
# Create a "basic" 1-layer LSTM model

LSTM1_model = keras.Sequential([
    keras.layers.Embedding(len(word_index)+1, 20, input_length=520),
    keras.layers.LSTM(100),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

LSTM1_model.compile(optimizer=keras.optimizers.Adam(learning_rate=hyper_params['LR']),
                 loss='binary_crossentropy', metrics=['accuracy'])

LSTM1_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 520, 20)           2244520   
                                                                 
 lstm (LSTM)                 (None, 100)               48400     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 2,293,021
Trainable params: 2,293,021
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Train the basic 1-layer LSTM model

LSTM1_history = LSTM1_model.fit(training_padded, train_labels, batch_size=hyper_params['BS'], 
                        epochs=hyper_params['EPOCHS'], validation_data=(testing_padded, test_labels))

Epoch 1/80
79/79 [==============================] - 7s 70ms/step - loss: 0.6935 - accuracy: 0.5024 - val_loss: 0.6935 - val_accuracy: 0.4997
Epoch 2/80
79/79 [==============================] - 5s 65ms/step - loss: 0.6976 - accuracy: 0.5108 - val_loss: 0.6968 - val_accuracy: 0.5077
Epoch 3/80
79/79 [==============================] - 5s 65ms/step - loss: 0.6772 - accuracy: 0.5208 - val_loss: 0.7052 - val_accuracy: 0.5071
Epoch 4/80
79/79 [==============================] - 5s 65ms/step - loss: 0.6658 - accuracy: 0.5290 - val_loss: 0.7103 - val_accuracy: 0.5039
Epoch 5/80
79/79 [==============================] - 5s 65ms/step - loss: 0.6531 - accuracy: 0.5352 - val_loss: 0.7162 - val_accuracy: 0.5011
Epoch 6/80
79/79 [==============================] - 5s 65ms/step - loss: 0.6472 - accuracy: 0.5396 - val_loss: 0.7394 - val_accuracy: 0.5060
Epoch 7/80
79/79 [==============================] - 5s 65ms/step - loss: 0.6446 - accuracy: 0.5425 - val_loss: 0.7743 - val_accuracy: 0.5051
Epoch 8/80
79

In [ ]:
# Save loss figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=LSTM1_history.epoch, y=LSTM1_history.history['loss'], name='Training Loss'))
fig.add_trace(go.Scatter(x=LSTM1_history.epoch, y=LSTM1_history.history['val_loss'], name='Validation Loss'))

fig.show()
fig.write_html('basic_LSTM1_loss.html')
#fig.write_image('basic_LSTM1_loss.jpeg')

In [ ]:
# Save accuracy figure

fig = go.Figure()
fig.add_trace(go.Scatter(x=LSTM1_history.epoch, y=LSTM1_history.history['accuracy'], name='Training Accuracy'))
fig.add_trace(go.Scatter(x=LSTM1_history.epoch, y=LSTM1_history.history['val_accuracy'], name='Validation Accuracy'))

fig.show()
fig.write_html('basic_LSTM1_acc.html')
#fig.write_image('basic_LSTM1_acc.jpeg')

In [ ]:
# Save history as csv and json

# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(LSTM1_history.history) 

# save to csv:
hist_csv_file = 'history_LSTM1_basic.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

#LSTM-2 RNN Model


We will construct a 2-layer LSTM to observe its behaviour. 

In [ ]:
# Create a "basic" 2-layer LSTM model

LSTM2_model = keras.Sequential([
    keras.layers.Embedding(len(word_index)+1, 20, input_length=520),
    keras.layers.LSTM(100, return_sequences=True),
    keras.layers.Dropout(0.5),
    keras.layers.LSTM(100),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

LSTM2_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.005),
                 loss='binary_crossentropy', metrics=['accuracy'])

LSTM2_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 520, 20)           2244520   
                                                                 
 lstm_1 (LSTM)               (None, 520, 100)          48400     
                                                                 
 dropout_4 (Dropout)         (None, 520, 100)          0         
                                                                 
 lstm_2 (LSTM)               (None, 100)               80400     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 101       
                                                                 
Total params: 2,373,421
Trainable params: 2,373,421
No

In [ ]:
# train the basic 2-layer LSTM model

LSTM2_history = LSTM2_model.fit(training_padded, train_labels, batch_size=hyper_params['BS'], 
                        epochs=hyper_params['EPOCHS'], validation_data=(testing_padded, test_labels))

Epoch 1/80
79/79 [==============================] - 15s 160ms/step - loss: 0.6937 - accuracy: 0.4993 - val_loss: 0.6932 - val_accuracy: 0.5024
Epoch 2/80
79/79 [==============================] - 12s 151ms/step - loss: 0.6915 - accuracy: 0.5062 - val_loss: 0.7056 - val_accuracy: 0.5061
Epoch 3/80
79/79 [==============================] - 12s 151ms/step - loss: 0.6749 - accuracy: 0.5267 - val_loss: 0.7048 - val_accuracy: 0.5023
Epoch 4/80
79/79 [==============================] - 12s 151ms/step - loss: 0.6516 - accuracy: 0.5384 - val_loss: 0.7712 - val_accuracy: 0.4983
Epoch 5/80
79/79 [==============================] - 12s 151ms/step - loss: 0.6449 - accuracy: 0.5408 - val_loss: 0.7554 - val_accuracy: 0.4983
Epoch 6/80
79/79 [==============================] - 12s 151ms/step - loss: 0.6433 - accuracy: 0.5364 - val_loss: 0.7680 - val_accuracy: 0.5015
Epoch 7/80
79/79 [==============================] - 12s 151ms/step - loss: 0.6423 - accuracy: 0.5425 - val_loss: 0.8030 - val_accuracy: 0.5000

In [ ]:
# Save loss figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=LSTM2_history.epoch, y=LSTM2_history.history['loss'], name='Training Loss'))
fig.add_trace(go.Scatter(x=LSTM2_history.epoch, y=LSTM2_history.history['val_loss'], name='Validation Loss'))

fig.show()
fig.write_html('basic_LSTM2_loss.html')
#fig.write_image('basic_LSTM2_loss.jpeg')

In [ ]:
# Save accuracy figure

fig = go.Figure()
fig.add_trace(go.Scatter(x=LSTM2_history.epoch, y=LSTM2_history.history['accuracy'], name='Training Accuracy'))
fig.add_trace(go.Scatter(x=LSTM2_history.epoch, y=LSTM2_history.history['val_accuracy'], name='Validation Accuracy'))

fig.show()
fig.write_html('basic_LSTM2_acc.html')
#fig.write_image('basic_LSTM2_acc.jpeg')

In [ ]:
# Save history as csv and json

# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(LSTM2_history.history) 

# save to csv:
hist_csv_file = 'history_LSTM2_basic.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)